In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="6"


In [4]:
#!/usr/bin/env python
# coding=utf-8
# Copyright The HuggingFace Team and The HuggingFace Inc. team. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""
Fine-tuning the library models for multiple choice.
"""
# You can also adapt this script on your own multiple choice task. Pointers for this are left as comments.

import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Optional, Union

import datasets
import numpy as np
import torch
from datasets import load_dataset

import transformers
from transformers import (
    AutoConfig,
    AutoModelForMultipleChoice,
    AutoTokenizer,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.file_utils import PaddingStrategy
from transformers.tokenization_utils_base import PreTrainedTokenizerBase
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version



logger = logging.getLogger(__name__)


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where do you want to store the pretrained models downloaded from huggingface.co"},
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": "Will use the token generated when running `transformers-cli login` (necessary to use this script "
            "with private models)."
        },
    )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    train_file: Optional[str] = field(default=None, metadata={"help": "The input training data file (a text file)."})
    validation_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input evaluation data file to evaluate the perplexity on (a text file)."},
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    max_seq_length: Optional[int] = field(
        default=None,
        metadata={
            "help": "The maximum total input sequence length after tokenization. If passed, sequences longer "
            "than this will be truncated, sequences shorter will be padded."
        },
    )
    pad_to_max_length: bool = field(
        default=False,
        metadata={
            "help": "Whether to pad all samples to the maximum sentence length. "
            "If False, will pad the samples dynamically when batching to the maximum length in the batch. More "
            "efficient on GPU but very bad for TPU."
        },
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of training examples to this "
            "value if set."
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
            "value if set."
        },
    )

    def __post_init__(self):
        if self.train_file is not None:
            extension = self.train_file.split(".")[-1]
            assert extension in ["csv", "jsonl"], "`train_file` should be a csv or a json file."
        if self.validation_file is not None:
            extension = self.validation_file.split(".")[-1]
            assert extension in ["csv", "jsonl"], "`validation_file` should be a csv or a json file."


@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    Args:
        tokenizer (:class:`~transformers.PreTrainedTokenizer` or :class:`~transformers.PreTrainedTokenizerFast`):
            The tokenizer used for encoding the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.file_utils.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "labels"
        # print(features[0].keys())
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        # Un-flatten
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add back labels
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch


In [5]:
parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
# if len(sys.argv) == 2 and sys.argv[1].endswith(".json"):
#     # If we pass only one argument to the script and it's the path to a json file,
#     # let's parse it to get our arguments.
model_args, data_args, training_args = parser.parse_json_file(json_file='run_guwen.json')

In [6]:
model_args

ModelArguments(model_name_or_path='ethanyt/guwenbert-base', config_name=None, tokenizer_name=None, cache_dir=None, use_fast_tokenizer=True, model_revision='main', use_auth_token=False)

In [7]:
data_args

DataTrainingArguments(train_file='data/train.jsonl', validation_file='data/valid.jsonl', overwrite_cache=False, preprocessing_num_workers=None, max_seq_length=None, pad_to_max_length=False, max_train_samples=None, max_eval_samples=None)

In [8]:
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)
log_level = training_args.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()

# Log on each process the small summary:
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f"distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
logger.info(f"Training/evaluation parameters {training_args}")

# Detecting last checkpoint.

11/25/2021 15:37:34 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
11/25/2021 15:37:34 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_e

In [9]:
last_checkpoint = None
if os.path.isdir(training_args.output_dir) and training_args.do_train and not training_args.overwrite_output_dir:
    last_checkpoint = get_last_checkpoint(training_args.output_dir)
    if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. "
            "Use --overwrite_output_dir to overcome."
        )
    elif last_checkpoint is not None and training_args.resume_from_checkpoint is None:
        logger.info(
            f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
            "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
        )

# Set seed before initializing model.
set_seed(training_args.seed)

# Get the datasets: you can either provide your own CSV/JSON/TXT training and evaluation files (see below)
# or just provide the name of one of the public datasets available on the hub at https://huggingface.co/datasets/
# (the dataset will be downloaded automatically from the datasets Hub).

# For CSV/JSON files, this script will use the column called 'text' or the first column if no column called
# 'text' is found. You can easily tweak this behavior (see below).

# In distributed training, the load_dataset function guarantee that only one local process can concurrently
# download the dataset.
if data_args.train_file is not None or data_args.validation_file is not None:
    data_files = {}
    if data_args.train_file is not None:
        data_files["train"] = data_args.train_file
    if data_args.validation_file is not None:
        data_files["validation"] = data_args.validation_file
    raw_datasets = load_dataset("json", data_files=data_files, cache_dir=model_args.cache_dir)
else:
    raise ValueError("train file and validation file are supposed to be specified")

# Load pretrained model and tokenizer

# Distributed training:
# The .from_pretrained methods guarantee that only one local process can concurrently
# download model & vocab.
config = AutoConfig.from_pretrained(
    model_args.config_name if model_args.config_name else model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
    revision=model_args.model_revision,
    use_auth_token=True if model_args.use_auth_token else None,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
    use_fast=model_args.use_fast_tokenizer,
    revision=model_args.model_revision,
    use_auth_token=True if model_args.use_auth_token else None,
)
model = AutoModelForMultipleChoice.from_pretrained(
    model_args.model_name_or_path,
    from_tf=bool(".ckpt" in model_args.model_name_or_path),
    config=config,
    cache_dir=model_args.cache_dir,
    revision=model_args.model_revision,
    use_auth_token=True if model_args.use_auth_token else None,
)

11/25/2021 15:37:38 - INFO - __main__ - Checkpoint detected, resuming training at results_o/checkpoint-4000. To avoid this behavior, change the `--output_dir` or add `--overwrite_output_dir` to train from scratch.
11/25/2021 15:37:53 - WARNING - datasets.builder - Using custom data configuration default-6032b753b6ad095d
11/25/2021 15:37:53 - INFO - datasets.builder - Overwrite dataset info from restored data version.
11/25/2021 15:37:53 - INFO - datasets.info - Loading Dataset info from /home/zhangkechi/.cache/huggingface/datasets/json/default-6032b753b6ad095d/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426
11/25/2021 15:37:53 - WARNING - datasets.builder - Reusing dataset json (/home/zhangkechi/.cache/huggingface/datasets/json/default-6032b753b6ad095d/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426)
11/25/2021 15:37:53 - INFO - datasets.info - Loading Dataset info from /home/zhangkechi/.cache/huggingface/datasets/json/default-6032b753b6ad0

  0%|          | 0/2 [00:00<?, ?it/s]

[INFO|configuration_utils.py:588] 2021-11-25 15:37:54,462 >> loading configuration file https://huggingface.co/ethanyt/guwenbert-base/resolve/main/config.json from cache at /home/zhangkechi/.cache/huggingface/transformers/1cd8aef41c425344f501c4072d646a95424c930385ee3568506cb54e66331d22.518e5ac3dc0dc1cb2c76aa391bc23d96ca5a5bc6b50df9ba9a411b3cd58f93d1
[INFO|configuration_utils.py:625] 2021-11-25 15:37:54,466 >> Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertTokenizer",
  "transformers_ve

In [10]:
o_embedding = model.roberta.embeddings.token_type_embeddings
n_embedding = torch.nn.Embedding(2,768)
n_embedding.weight = torch.nn.Parameter(o_embedding.weight.repeat(2,1))
model.roberta.embeddings.token_type_embeddings = n_embedding

In [11]:
ending_names = [f"ending{i}" for i in range(4)]
context_name = "translation"
choice_name = "choices"

if data_args.max_seq_length is None:
    max_seq_length = tokenizer.model_max_length
    if max_seq_length > 1024:
        logger.warning(
            f"The tokenizer picked seems to have a very large `model_max_length` ({tokenizer.model_max_length}). "
            "Picking 1024 instead. You can change that default value by passing --max_seq_length xxx."
        )
        max_seq_length = 1024
else:
    if data_args.max_seq_length > tokenizer.model_max_length:
        logger.warning(
            f"The max_seq_length passed ({data_args.max_seq_length}) is larger than the maximum length for the"
            f"model ({tokenizer.model_max_length}). Using max_seq_length={tokenizer.model_max_length}."
        )
    max_seq_length = min(data_args.max_seq_length, tokenizer.model_max_length)

# Preprocessing the datasets.
def preprocess_function(examples):
    translation = [[context] * 4 for context in examples[context_name]]
    classic_poetry = [
        [c for c in choices] for choices in examples[choice_name]
    ]

    # Flatten out
    first_sentences = sum(translation, [])
    second_sentences = sum(classic_poetry, [])

    # Tokenize
    tokenized_examples = tokenizer(
        first_sentences,
        second_sentences,
        truncation=True,
        max_length=max_seq_length,
        padding="max_length" if data_args.pad_to_max_length else False,
    )
    results = {}
    results.update({k: [v[i : i + 4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()})
    results['labels'] = [ answer for answer in examples['answer']]
    # print(results)
    # Un-flatten
    return results 


if training_args.do_train:
    if "train" not in raw_datasets:
        raise ValueError("--do_train requires a train dataset")
    train_dataset = raw_datasets["train"]
    if data_args.max_train_samples is not None:
        train_dataset = train_dataset.select(range(data_args.max_train_samples))
    with training_args.main_process_first(desc="train dataset map pre-processing"):
        train_dataset = train_dataset.map(
            preprocess_function,
            batched=True,
            num_proc=data_args.preprocessing_num_workers,
            load_from_cache_file=not data_args.overwrite_cache,
        )

if training_args.do_eval:
    if "validation" not in raw_datasets:
        raise ValueError("--do_eval requires a validation dataset")
    eval_dataset = raw_datasets["validation"]
    if data_args.max_eval_samples is not None:
        eval_dataset = eval_dataset.select(range(data_args.max_eval_samples))
    with training_args.main_process_first(desc="validation dataset map pre-processing"):
        eval_dataset = eval_dataset.map(
            preprocess_function,
            batched=True,
            num_proc=data_args.preprocessing_num_workers,
            load_from_cache_file=not data_args.overwrite_cache,
        )

# Data collator
data_collator = (
    default_data_collator
    if data_args.pad_to_max_length
    else DataCollatorForMultipleChoice(tokenizer=tokenizer, pad_to_multiple_of=8 if training_args.fp16 else None)
)

# Metric
def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}


11/25/2021 15:38:28 - WARNING - __main__ - The tokenizer picked seems to have a very large `model_max_length` (1000000000000000019884624838656). Picking 1024 instead. You can change that default value by passing --max_seq_length xxx.
11/25/2021 15:38:28 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /home/zhangkechi/.cache/huggingface/datasets/json/default-6032b753b6ad095d/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426/cache-2a6a97dc2d3e3b09.arrow
11/25/2021 15:38:28 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /home/zhangkechi/.cache/huggingface/datasets/json/default-6032b753b6ad095d/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426/cache-68a4acd445f77f09.arrow


In [12]:
model

RobertaForMultipleChoice(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(23292, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): La

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset if training_args.do_train else None,
    eval_dataset=eval_dataset if training_args.do_eval else None,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

[INFO|trainer.py:434] 2021-11-25 15:38:47,872 >> Using amp fp16 backend


In [15]:
train_dataloader = trainer.get_train_dataloader()

[INFO|trainer.py:541] 2021-11-25 15:38:50,693 >> The following columns in the training set  don't have a corresponding argument in `RobertaForMultipleChoice.forward` and have been ignored: translation, choices, answer.


In [16]:
model = model.cpu()
for i, b in enumerate(train_dataloader):
    out = model(**b)
    break

In [22]:
raw_datasets['train'][0]

{'translation': '诗人啊，你竟像在遥远的地方站立船头。',
 'choices': ['行人初上木兰舟', '骚人遥驻木兰舟', '有人独上木兰舟', '行人迢递木兰舟'],
 'answer': 1}

In [51]:
def my_preprocess_function(examples):
    # print(len(examples['translation']))
    def split_string(c,l = 5):
        if l == 5:
            return [c[:2],c[2:5]]
        elif l == 7:
            return [c[:2],c[2:4],c[4:]]
        elif l == 12:
            return [c[:2],c[2:5],c[6:8],c[8:11]]
        elif l == 16:
            return [c[:2],c[2:4],c[4:7],c[8:10],c[10:12],c[12:15]]
    def generate_translation_and_classic_span(examples):
        '''
        7 行人初上木兰舟
        16 绿玉觜攒鸡脑破，玄金爪擘兔心开。
        5 清晨西北转
        12 篱落深村路，闾阎处士家。
        '''
        translation = []
        classic_span = []
        labels = []
        for i in range(len(examples['choices'])):
            this_choices = []
            choice_len = len(examples['choices'][i][0])
            for c in examples['choices'][i]:
                this_choices.extend(split_string(c,choice_len))
            
            this_answer = examples['answer'][i]
            this_answer = examples['choices'][i][this_answer]
            true_spans = split_string(this_answer, choice_len)

            this_choices = list(set(this_choices))
            this_labels = [int(e in true_spans) for e in this_choices]

            this_translations = [examples['translation'][i]] * len(this_choices)

            translation.extend(this_translations)
            classic_span.extend(this_choices)
            labels.extend(this_labels)
        return translation, classic_span, labels


    translation, classic_span, labels = generate_translation_and_classic_span(examples)

    # Flatten out
    first_sentences = translation
    second_sentences = classic_span

    # print(first_sentences)
    # print(second_sentences)
    # assert(False)

    # Tokenize
    # print(len(first_sentences))
    # print(first_sentences)
    # print(second_sentences)
    tokenized_examples = tokenizer(
        first_sentences,
        second_sentences,
        truncation=True,
        max_length=max_seq_length,
        padding="max_length" if data_args.pad_to_max_length else False,
    )
    # print(tokenized_examples.keys())
    results = {}
    results.update(tokenized_examples)
    results['translation'] = translation
    results['classic_poetry'] = classic_span
    results['labels'] = labels
    # print(results)
    # Un-flatten
    return results 
my_dataset = raw_datasets['train'].map(
                my_preprocess_function,
                batched=True,remove_columns=raw_datasets['train'].column_names)

my_dataset['translation'][:10]

  0%|          | 0/22 [00:00<?, ?ba/s]

11/25/2021 16:08:50 - INFO - datasets.arrow_dataset - Caching processed dataset at /home/zhangkechi/.cache/huggingface/datasets/json/default-6032b753b6ad095d/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426/cache-654ad50a99b008fe.arrow


['诗人啊，你竟像在遥远的地方站立船头。',
 '诗人啊，你竟像在遥远的地方站立船头。',
 '诗人啊，你竟像在遥远的地方站立船头。',
 '诗人啊，你竟像在遥远的地方站立船头。',
 '诗人啊，你竟像在遥远的地方站立船头。',
 '诗人啊，你竟像在遥远的地方站立船头。',
 '诗人啊，你竟像在遥远的地方站立船头。',
 '诗人啊，你竟像在遥远的地方站立船头。',
 '他的双眼眼瞳碧绿而有光，头发金黄而弯曲，两鬓呈红色。',
 '他的双眼眼瞳碧绿而有光，头发金黄而弯曲，两鬓呈红色。']

In [60]:
my_dataset['labels'][:10]

[0, 1, 1, 0, 0, 0, 0, 1, 1, 0]

In [62]:
my_dataset['translation'][:10]

['诗人啊，你竟像在遥远的地方站立船头。',
 '诗人啊，你竟像在遥远的地方站立船头。',
 '诗人啊，你竟像在遥远的地方站立船头。',
 '诗人啊，你竟像在遥远的地方站立船头。',
 '诗人啊，你竟像在遥远的地方站立船头。',
 '诗人啊，你竟像在遥远的地方站立船头。',
 '诗人啊，你竟像在遥远的地方站立船头。',
 '诗人啊，你竟像在遥远的地方站立船头。',
 '他的双眼眼瞳碧绿而有光，头发金黄而弯曲，两鬓呈红色。',
 '他的双眼眼瞳碧绿而有光，头发金黄而弯曲，两鬓呈红色。']

In [25]:
len(my_dataset['translation']),len(my_dataset['classic_poetry'])

(21778, 21778)

In [23]:
len_set = []
for e in my_dataset['classic_poetry']:
    if len(e[0]) not in len_set:
        print(len(e[0]), e[0])
        len_set.append(len(e[0]))

7 行人初上木兰舟
16 绿玉觜攒鸡脑破，玄金爪擘兔心开。
5 清晨西北转
12 篱落深村路，闾阎处士家。


In [25]:
for k in b:
    print(k)
    print(b[k].shape)
print(b['labels'])
print(out.logits.shape)

attention_mask
torch.Size([16, 4, 56])
input_ids
torch.Size([16, 4, 56])
token_type_ids
torch.Size([16, 4, 56])
labels
torch.Size([16])
tensor([0, 1, 3, 2, 1, 1, 2, 0, 3, 1, 0, 3, 3, 1, 1, 3])
torch.Size([16, 4])


In [27]:
type(model)

transformers.models.roberta.modeling_roberta.RobertaForMultipleChoice

In [31]:
attention_mask = b['attention_mask']
input_ids = b['input_ids']
token_type_ids = b['token_type_ids']
labels = b['labels']
flat_input_ids = input_ids.view(-1, input_ids.size(-1)) if input_ids is not None else None
flat_token_type_ids = token_type_ids.view(-1, token_type_ids.size(-1)) if token_type_ids is not None else None
flat_attention_mask = attention_mask.view(-1, attention_mask.size(-1)) if attention_mask is not None else None

In [32]:
roberta_output = model.roberta(flat_input_ids,token_type_ids=flat_token_type_ids, attention_mask=flat_attention_mask)
roberta_output[1].shape

In [55]:
logits = model.classifier(roberta_output[1])
logits.shape

torch.Size([64, 1])

In [ ]:
# When using your own dataset or a different dataset from swag, you will probably need to change this.

# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset if training_args.do_train else None,
    eval_dataset=eval_dataset if training_args.do_eval else None,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Training
if training_args.do_train:
    checkpoint = None
    if training_args.resume_from_checkpoint is not None:
        checkpoint = training_args.resume_from_checkpoint
    elif last_checkpoint is not None:
        checkpoint = last_checkpoint
    train_result = trainer.train(resume_from_checkpoint=checkpoint)
    trainer.save_model()  # Saves the tokenizer too for easy upload
    metrics = train_result.metrics

    max_train_samples = (
        data_args.max_train_samples if data_args.max_train_samples is not None else len(train_dataset)
    )
    metrics["train_samples"] = min(max_train_samples, len(train_dataset))

    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()

# Evaluation
if training_args.do_eval:
    logger.info("*** Evaluate ***")

    metrics = trainer.evaluate()
    max_eval_samples = data_args.max_eval_samples if data_args.max_eval_samples is not None else len(eval_dataset)
    metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))

    trainer.log_metrics("eval", metrics)
    trainer.save_metrics("eval", metrics)